In [11]:
from keras.models import Sequential, clone_model, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, TimeDistributed, Conv2D, Reshape
from keras.layers import Convolution2D, MaxPooling2D, Concatenate, CuDNNLSTM, Cropping1D
from keras.losses import mean_squared_error
from keras.optimizers import Adam
from keras.utils import Sequence
import numpy as np
import tensorflow as tf
from keras import backend as K
# tf.reset_default_graph()
TD = TimeDistributed
sess = tf.Session()
K.set_session(sess)

In [12]:
batch_size = 32
learning_rate = 0.001
T_in = 16
T_total = 100
IMG_H = 125
IMG_W = 125
IMG_CH = 3

In [13]:
def build_autoencoder():
  inputs = Input(shape=(T_in, IMG_H, IMG_W, IMG_CH))
  # Apply convolutions on the initial image input with increasing channel size.
  conv_x = TD(Conv2D(32, kernel_size=(3, 3), strides=(2, 2), activation='relu'))(inputs)
  conv_x = TD(MaxPooling2D(pool_size=(2, 2)))(conv_x)
  conv_x = TD(Conv2D(64, kernel_size=(3, 3), strides=(2, 2), activation='relu'))(conv_x)
  conv_x = TD(MaxPooling2D(pool_size=(2, 2)))(conv_x)
  conv_x = TD(Conv2D(128, kernel_size=(3, 3), strides=(2, 2), activation='relu'))(conv_x)
  print("Exit initial conv: ", conv_x.shape)

  # We pass the flattened convolution output into a CuDNN-optimized LSTM.
  # Outputs are disregarded for training but form the "encoded" representation.
  enc_x = Reshape((T_in, -1))(conv_x)
  encoded = Concatenate()(CuDNNLSTM(1024, return_state=True, return_sequences=False)(enc_x))
  return Model(inputs=inputs, outputs=encoded)

autoencoder = build_autoencoder()
autoencoder.load_weights("/home/ubuntu/semisupervised_mil/autoencoder/model_weights_6.h5", by_name=True)

Exit initial conv:  (?, 16, 3, 3, 128)


ValueError: Layer #12 (named "time_distributed_8") expects 2 weight(s), but the saved weights have 0 element(s).

In [ ]:
def build_worse_model():
  inputs = Input(shape=(T_in, IMG_H, IMG_W, IMG_CH))
  model = TD(Flatten())(inputs)
  model = Cropping1D(cropping=(T_in - 1, 0))(model)
  model = Reshape((IMG_H, IMG_W, IMG_CH))(model)
  model = Convolution2D(32, (3, 3), padding='same', activation="relu")(model)
  model = MaxPooling2D(pool_size=(3, 3))(model)
  model = Convolution2D(32, (3, 3), padding='same', activation="relu")(model)
  model = MaxPooling2D(pool_size=(3, 3))(model)
  print(model.shape)
  model = Dropout(0.25)(model)
  model = Flatten()(model)
  model = Dense(512, activation="relu")(model)
  model = Dropout(0.5)(model)
  preds = Dense(7, activation="tanh")(model)
  
  keras_model = Model(inputs=inputs, outputs=preds)
  
  return keras_model

In [ ]:
def build_better_model():
  inputs = Input(shape=(T_in, IMG_H, IMG_W, IMG_CH))
  model = autoencoder(inputs)
  model = Dense(512, activation="relu")(model)
  model = Dropout(0.5)(model)
  preds = Dense(7, activation="tanh")(model)
  keras_model = Model(inputs=inputs, outputs=preds)
  
  return keras_model

In [14]:
# REMEMBER TO UPDATE
build_model = build_better_model

In [15]:
# Load train data
train_input = tf.placeholder(tf.float32, shape=(batch_size, T_in, IMG_H, IMG_W, IMG_CH))
train_label = tf.placeholder(tf.float32, shape=(batch_size, 7))
# Build initial model
before_model = build_model()
# Calulate initial loss
before_pred = before_model(train_input)
# Calculate loss and gradient for the task
before_loss = tf.reduce_mean(mean_squared_error(train_label, before_pred))
before_gradients = tf.gradients(before_loss, before_model.trainable_weights)
# Calculate ethereal weights for task-specific network
ethereal = {}
for weight, gradient in zip(before_model.trainable_weights, before_gradients):
  ethereal[weight] = weight - learning_rate * gradient
# Load test data
specialized_copy_ops = []
test_input = tf.placeholder(tf.float32, shape=(batch_size, T_in, IMG_H, IMG_W, IMG_CH))
test_label = tf.placeholder(tf.float32, shape=(batch_size, 7))
# Build new ethereal model
after_model = build_model()
for before_weight, after_weight in zip(before_model.trainable_weights,
                                       after_model.trainable_weights):
  specialized_copy_ops.append(tf.assign(after_weight, ethereal[before_weight]))
# Calculate the final gradients!
after_pred = after_model(test_input)
after_loss = tf.reduce_mean(mean_squared_error(test_label, after_pred))
after_grads = tf.gradients(after_loss, after_model.trainable_weights)

# Update our meta weights!
meta_ops = []
for before_weight, grad in zip(before_model.trainable_weights, after_grads):
  meta_ops.append(tf.assign(before_weight, before_weight - learning_rate * grad))

In [16]:
def train(train_x, train_y, test_x, test_y):
  sess.run(specialized_copy_ops, feed_dict={train_input: train_x,
                                            train_label: train_y})
  sess.run(meta_ops, feed_dict={test_input: test_x,
                                test_label: test_y})
  
def test(train_x, train_y, test_x, test_y):
  sess.run(specialized_copy_ops, feed_dict={train_input: train_x,
                                            train_label: train_y})
  return sess.run(after_loss, feed_dict={test_input: test_x,
                                         test_label: test_y})

In [17]:
# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [ ]:
epochs = 1000
for i in range(epochs):
  for train_x, train_y, test_x, test_y in get_batch():
    for i in range(15, T_total):
      train(train_x[:,i-15:i+1], train_y[:,i], test_x[:,i-15:i+1], test_y[:,i])
      print(test(train_x[:,i-15:i+1], train_y[:,i], test_x[:,i-15:i+1], test_y[:,i]))

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
show_graph(tf.get_default_graph().as_graph_def())

In [18]:
train_x = np.random.uniform(size=(batch_size, T_total, IMG_H, IMG_W, IMG_CH))
train_y= np.random.uniform(size=(batch_size, T_total, 7))
test_x = np.random.uniform(size=(batch_size, T_total, IMG_H, IMG_W, IMG_CH))
test_y = np.random.uniform(size=(batch_size, T_total, 7))
for i in range(15, T_total):
  train(train_x[:,i-15:i+1], train_y[:,i], test_x[:,i-15:i+1], test_y[:,i])
  print(test(train_x[:,i-15:i+1], train_y[:,i], test_x[:,i-15:i+1], test_y[:,i]))

0.29483178
0.27438277
0.2569144
0.2352502


KeyboardInterrupt: 